# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Unleashing a New Era of AI With DeepLearning.AI"

Discover the powerhouse behind cutting-edge AI innovation: DeepLearning.AI. Founded by the world-renowned AI expert, Andrew Ng, this platform provides top-notch AI education to aspirants worldwide. Offering specialized courses in deep learning, machine learning, and AI, DeepLearning.AI equips learners with the skills needed to thrive in the AI industry. With a user-friendly interface and interactive learning experiences, this platform is revolutionizing the way people engage with AI education. Embrace the future of technology and dive into the world of artificial intelligence with DeepLearning.AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing a New Era of AI With DeepLearning.AI"

Discover the powerhouse behind cutting-edge AI innovation: DeepLearning.AI. Founded by the world-renowned AI expert, Andrew Ng, this platform provides top-notch AI education to aspirants worldwide. Offering specialized courses in deep learning, machine learning, and AI, DeepLearning.AI equips learners with the skills needed to thrive in the AI industry. With a user-friendly interface and interactive learning experiences, this platform is revolutionizing the way people engage with AI education. Embrace the future of technology and dive into the world of artificial intelligence with DeepLearning.AI.

----------------------------------------------------------------------------

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing a New Era of AI With DeepLearning.AI"

Discover the powerhouse behind cutting-edge AI innovation: DeepLearning.AI. Founded by the world-renowned AI expert, Andrew Ng, this platform provides top-notch AI education to aspirants worldwide. Offering specialized courses in deep learning, machine learning, and AI, DeepLearning.AI equips learners with the skills needed to thrive in the AI industry. With a user-friendly interface and interactive learning experiences, this platform is revolutionizing the way people engage with AI education. Embrace the future of technology and dive into the world of artificial intelligence with DeepLearning.AI.

----------------------------------------------------------------------------


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregated feedback from all reviewers:

1. SEO Reviewer:
- Incorporate relevant keywords like "AI education," "deep learning courses," and "Andrew Ng" to improve search engine visibility.
- Include meta title and description tags with targeted keywords for better click-through rates.
- Add internal links to related content for enhanced SEO and user engagement.

2. Legal Reviewer:
- Ensure compliance with intellectual property rights.
- Verify accuracy of claims.
- Review for false advertising or misleading statements.

3. Ethics Reviewer:
- Maintain transparency regarding any biases or financial relationships with DeepLearning.AI.
- Disclose conflicts of interest that may affect the promotion of the platform.
- Review content for exaggerated claims about the effectiveness of AI courses.
- Avoid creating unrealistic expectations or pressures related to AI education.

Final sugg

## Get the summary

In [15]:
print(res.summary)

Title: "Empowering Minds: Exploring the World of DeepLearning.AI"

Delve into the realm of AI education with DeepLearning.AI, spearheaded by the eminent AI guru, Andrew Ng. Unleashing a treasure trove of deep learning courses, this platform equips learners worldwide with the prowess to excel in the AI domain. Navigate through interactive modules and user-friendly interfaces designed to revolutionize your AI learning experience. From machine learning to artificial intelligence, DeepLearning.AI offers a gateway to the future of technology. Embrace this transformative journey and unravel the endless possibilities that AI education has to offer.
